In [1]:
import pandas as pd
import geopandas as gpd
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
import geopandas as gpd
import pysal as ps

In [2]:
import os
datapath ="C:/Users/MISBAHUL HAQUE/Desktop/Shape_WB"
grid = gpd.read_file(os.path.join(datapath,'Final.shp'))
df1=pd.read_excel(os.path.join(r'C:\Users\MISBAHUL HAQUE\Desktop','Merged_book.xlsx'))

df=pd.read_excel(os.path.join(r'C:\Users\MISBAHUL HAQUE\Desktop','result.xlsx'))


df2=pd.read_excel(os.path.join(r'C:\Users\MISBAHUL HAQUE\Desktop','result.xlsx'))
df_merge_col = pd.merge(grid, df1, on='Block', how='left')
grid=pd.merge(grid,df2, on='Block', how='left')

grid.head()

,Id_x,Block,geometry,Unnamed: 0,Unnamed: 0.1,Id_y,BLOCK,idcode
0,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",0.0,0.0,0.0,SAGAR,118529.0
1,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",1.0,1.0,0.0,SAGAR,118529.0
2,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",7.0,7.0,0.0,SAGAR,118529.0
3,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",8.0,8.0,0.0,SAGAR,118529.0
4,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",17.0,17.0,0.0,SAGAR,118529.0


In [3]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [4]:
df_wheat=pd.read_excel(os.path.join(r'C:\Users\MISBAHUL HAQUE\Desktop','Wheat.xls'))
df_wheat=pd.DataFrame(df_wheat,columns=['YR','idcode','blkdryyr','Hol','Result'])
df_wheat.head()

,YR,idcode,blkdryyr,Hol,Result
0,1000,1911610406,1457.39,NaN,Bad
1,1000,1911620606,987.78,NaN,Bad
2,1000,1911620706,1898.43,NaN,Bad
3,1000,1911620806,2081.96,NaN,Bad
4,4000,1920810106,2842.36,NaN,Good


In [5]:

df_wheat.dtypes


YR            int64
idcode        int64
blkdryyr    float64
Hol         float64
Result       object
dtype: object

In [6]:
df_wheat[["idcode"]]=df_wheat[["idcode"]].astype(str)

In [7]:
df_wheat.dtypes

YR            int64
idcode       object
blkdryyr    float64
Hol         float64
Result       object
dtype: object

In [8]:
df_wheat['idcode'] = df_wheat['idcode'].str[2:8]

In [9]:
df_wheat.head()
df_wheat[["idcode"]]=df_wheat[["idcode"]].astype(int)

In [10]:
df_wheat.to_excel(r"C:\Users\MISBAHUL HAQUE\Desktop\WB\Wheat.xlsx")
df_block=pd.read_excel(os.path.join(r'C:\Users\MISBAHUL HAQUE\Desktop','Wht_hol.xlsx'))
df_block[["Value"]]=df_block[["Value"]].astype(float)
df_merge = pd.merge(grid,df_wheat, on='idcode', how='left')
df_merge.dtypes

#adding the wheat holiday blocks to  new dataframe

df_hol=pd.merge(df_block,grid, on='Block', how='left')
df_merge.head()

,Id_x,Block,geometry,Unnamed: 0,Unnamed: 0.1,Id_y,BLOCK,idcode,YR,blkdryyr,Hol,Result
0,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",0.0,0.0,0.0,SAGAR,118529.0,NaN,NaN,NaN,NaN
1,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",1.0,1.0,0.0,SAGAR,118529.0,NaN,NaN,NaN,NaN
2,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",7.0,7.0,0.0,SAGAR,118529.0,NaN,NaN,NaN,NaN
3,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",8.0,8.0,0.0,SAGAR,118529.0,NaN,NaN,NaN,NaN
4,0,Sagar,"POLYGON ((159.5191040039063 45.731689453125, 1...",17.0,17.0,0.0,SAGAR,118529.0,NaN,NaN,NaN,NaN


In [11]:

df_merge['x'] = df_merge.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
df_merge['y'] = df_merge.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

df_hol['x'] = df_hol.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
df_hol['y'] = df_hol.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

classifier = ps.Quantiles.make(k=6)
from bokeh.palettes import RdBu3

g_df = df_merge.drop('geometry', axis=1).copy()
g_df1=df_hol.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)
gs1=ColumnDataSource(g_df1)


In [12]:
from bokeh.palettes import Paired10 as palette
from bokeh.palettes import Greys256 as pal
from bokeh.models import LogColorMapper, LogTicker, ColorBar

color_mapper = LogColorMapper(palette=palette)
color_mapper1 = LogColorMapper(palette=pal)

palette.reverse()


In [13]:
p = figure(title="WHEAT YIELD ANALYSIS 2018-19",plot_width=590,plot_height=760,background_fill_color="White")

p.title.align="right"
p.title.text_color="yellow"
p.title.text_font_size="20px"


pal.reverse()
palette.reverse()


p.patches('x', 'y', source=gsource,
         fill_color={'field': 'YR', 'transform': color_mapper},
         fill_alpha=1.25, line_color="black", line_width=0.50)

p.patches('x', 'y', source=gs1,
         fill_color={'field': 'Value', 'transform': color_mapper1},
         fill_alpha=1.25, line_color="white", line_width=0.25)
p.grid.grid_line_color = None

#  hover over info tool
tooltip = HoverTool()
tooltip.tooltips = [('Name  of the Block', '@Block'),
                    ('Yield(Kgs/Hectare)','@blkdryyr'),('Wheat Holiday','@Hol'),('Result','@Result')]
p.add_tools(tooltip)



In [14]:
from bokeh.models import Legend, LegendItem
from bokeh.models import LogColorMapper, LogTicker, ColorBar
from bokeh.transform import linear_cmap

custom_pal=['#FF0000','#0000FF','#008000']

mapper = linear_cmap(field_name=df_merge['blkdryyr'], palette="Paired10" ,high=4400, low=1800)

color_bar = ColorBar(color_mapper=mapper['transform'],  width=15,  location=(0,0))

p.add_layout(color_bar, 'right')

from bokeh.models.widgets import TextAreaInput
text=TextAreaInput(value="default", rows=6, title="Conditions:")

from bokeh.models import Panel, Tabs

tab1=Panel(child=p,title='Yield Rate<2400 : BAD')
tab2=Panel(child=p,title='Yield Rate>2400 but <2800 : Normal')
tab3=Panel(child=p,title='Yield Rate>2800 : GOOD')

tab=Tabs(tabs=[tab1,tab2,tab3])


In [15]:
from bokeh.plotting import figure, show, output_file, output_notebook
outfp = r"C:\Users\MISBAHUL HAQUE\Desktop\WB\Wheat_2018-19.html"
output_file(outfp)
show(tab)
